In [1]:
import numpy as np
import pandas as pd
from xgboost import *
from sklearn.metrics import accuracy_score
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.preprocessing import LabelEncoder
from sklearn.grid_search import GridSearchCV
import re
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics

/Users/gongtingpeng/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/gongtingpeng/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [73]:
train = pd.read_feather('tmp/train-lat-long')
test = pd.read_feather('tmp/test-lat-long')

In [74]:
category = ["Street", "From", "To","from_to"]

le = LabelEncoder()
for cat in category:
    le.fit(train[cat])
    train[cat] = le.transform(train[cat])
    test[cat] = le.transform(test[cat])

In [3]:
# #Handles class imbalance through sampling
# classDistribution = train['any_spot'].value_counts()
# print('Class imbalance:')
# print(classDistribution)
# train = train.sample(frac=1).reset_index(drop=True)
# zero = train[train['any_spot']==0].tail(classDistribution.min())
# one = train[train['any_spot']==1]
# train_data = zero.append(one)
# del zero, one
# train_data = train_data.sample(frac=1).reset_index(drop=True)
# #del train
# print('Class imbalance evened out:')
# print(train_data['any_spot'].value_counts())

In [75]:
X = train.drop(['any_spot', 'Real.Spots'], axis=1)
Y = train['any_spot']

In [63]:
train

,Street,From,To,Real.Spots,Street.Length,any_spot,from_to,STREET_NAME_x,occ_per_1,vac_per_1,...,mean_isweekday_real,mean_hr_any,mean_hr_real,mean_dow_any,mean_dow_real,mean_month_any,mean_month_real,combined,lat_long_from,lat_long_to
0,15,1,1,4,179.132970,1,1,MISSION STREET,0.551448,0.414499,...,1.497872,0.428571,1.551020,0.329787,1.276596,0.502924,3.912281,"MISSION STREET,25th Street,26th Street","lat:37.7506482,lng:-122.4183159","lat:37.749044,lng:-122.4181659"
1,18,16,42,0,52.740210,0,22,POLK STREET,0.435176,0.473185,...,2.134177,0.236559,1.000000,0.415493,2.102113,0.502924,3.912281,"POLK STREET,Ellis Street,Olive Street","lat:37.7840397,lng:-122.4193807","lat:37.784507,lng:-122.4194781"
2,24,17,40,0,52.517840,0,23,VAN NESS AVENUE,0.353977,0.645017,...,2.134177,0.236559,1.000000,0.415493,2.102113,0.502924,3.912281,"VAN NESS AVENUE,Geary Boulevard,Myrtle Street","lat:37.7856601,lng:-122.4214764","lat:37.7852409,lng:-122.421234"
3,24,8,21,0,52.405315,0,10,VAN NESS AVENUE,0.353977,0.645017,...,2.134177,0.236559,1.000000,0.415493,2.102113,0.502924,3.912281,"VAN NESS AVENUE,Bush Street,Fern Street","lat:37.7884986,lng:-122.4218922","lat:37.7880441,lng:-122.4218018"
4,24,13,46,0,52.191193,0,17,VAN NESS AVENUE,0.353977,0.645017,...,2.134177,0.236559,1.000000,0.415493,2.102113,0.502924,3.912281,"VAN NESS AVENUE,Daniel Burnham Court,Post Street","lat:37.7870844,lng:-122.4217571","lat:37.7866423,lng:-122.4215171"
5,12,10,46,0,52.324250,0,13,LARKIN STREET,0.388997,0.454687,...,2.134177,0.236559,1.000000,0.415493,2.102113,0.502924,3.912281,"LARKIN STREET,Cedar Street,Post Street","lat:37.7865918,lng:-122.418214","lat:37.7870509,lng:-122.4183057"
6,17,30,45,0,146.416550,0,43,PINE STREET,0.384777,0.469562,...,2.134177,0.236559,1.000000,0.415493,2.102113,0.502924,3.912281,"PINE STREET,Larkin Street,Polk Street","lat:37.7898426,lng:-122.4188741","lat:37.7896366,lng:-122.4205069"
7,17,43,55,0,139.494200,0,62,PINE STREET,0.384777,0.469562,...,2.134177,0.236559,1.000000,0.415493,2.102113,0.502924,3.912281,"PINE STREET,Polk Street,Van Ness Avenue","lat:37.7896366,lng:-122.4205069","lat:37.7894225,lng:-122.4220832"
8,12,37,23,0,52.188343,0,53,LARKIN STREET,0.388997,0.454687,...,2.134177,0.236559,1.000000,0.415493,2.102113,0.502924,3.912281,"LARKIN STREET,Myrtle Street,Geary Street","lat:37.7856499,lng:-122.4180244","lat:37.786119,lng:-122.4181182"
9,24,52,9,0,39.027992,0,76,VAN NESS AVENUE,0.353977,0.645017,...,2.134177,0.236559,1.000000,0.415493,2.102113,0.502924,3.912281,"VAN NESS AVENUE,Van Ness Avenue,Bush Street","lat:37.7752197,lng:-122.4192105","lat:37.7884986,lng:-122.4218922"


In [26]:
X.columns

Index(['Street', 'From', 'To', 'Street.Length', 'from_to', 'STREET_NAME_x',
       'occ_per_1', 'vac_per_1', 'occ_per_2', 'vac_per_2', 'occ_per_3',
       'vac_per_3', 'occ_per_4', 'vac_per_4', 'STREET_NAME_y', 'count',
       'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'hr', 'min',
       'isweekday', 'mean_from_to_any', 'mean_from_to_real',
       'mean_isweekday_any', 'mean_isweekday_real', 'mean_hr_any',
       'mean_hr_real', 'mean_dow_any', 'mean_dow_real', 'mean_month_any',
       'mean_month_real', 'combined', 'lat_long_from', 'lat_long_to'],
      dtype='object')

In [76]:
X['lat_from'], X['lon_from'] = X.lat_long_from.str.split(',').str

_, X['lat_from'] = X.lat_from.str.split(':').str

_, X['lon_from'] = X.lon_from.str.split(':').str

X = X.drop(['lat_long_from'], axis = 1)

X['lat_to'], X['lon_to'] = X.lat_long_to.str.split(',').str

_, X['lat_to'] = X.lat_to.str.split(':').str

_, X['lon_to'] = X.lon_to.str.split(':').str

X = X.drop(['lat_long_to'], axis = 1)

In [77]:
X = X.drop(['STREET_NAME_x', 'STREET_NAME_y'], axis = 1)
test = test.drop(['STREET_NAME_x', 'STREET_NAME_y'], axis = 1)

In [69]:
test.head()

,Street,From,To,Street.Length,from_to,occ_per_1,vac_per_1,occ_per_2,vac_per_2,occ_per_3,...,mean_isweekday_real,mean_hr_any,mean_hr_real,mean_dow_any,mean_dow_real,mean_month_any,mean_month_real,lat_long_from,lat_to,lon_to
0,21,48,48,35.786472,68,0.759830,0.239126,0.867197,0.131638,0.652784,...,1.497872,0.428571,1.551020,0.292576,0.812227,0.321117,1.235602,"lat:37.7927714,lng:-122.4076486",37.7940644,-122.4079215
1,24,24,55,63.787968,31,0.353977,0.645017,0.447894,0.550980,0.357339,...,1.497872,0.285714,0.659341,0.292576,0.812227,0.321117,1.235602,"lat:37.7773125,lng:-122.4196331",37.7917022,-122.4226899
2,24,34,55,56.007236,48,0.353977,0.645017,0.447894,0.550980,0.357339,...,1.497872,0.290323,1.118280,0.292576,0.812227,0.321117,1.235602,"lat:37.7800953,lng:-122.4201902",37.7810463,-122.4203923
3,15,0,38,139.651900,0,0.551448,0.414499,0.673388,0.321598,0.499000,...,1.497872,0.236559,1.000000,0.292576,0.812227,0.321117,1.235602,"lat:37.7743325,lng:-122.4171372",37.7730672,-122.4187201
4,7,19,35,105.144110,25,0.506700,0.369059,0.688952,0.186181,0.325301,...,1.497872,0.290323,1.118280,0.292576,0.812227,0.321117,1.235602,"lat:37.7816678,lng:-122.4155242",37.7807062,-122.4153337


In [78]:
test['lat_from'], test['lon_from'] = test.lat_long_from.str.split(',').str

_, test['lat_from'] = test.lat_from.str.split(':').str

_, test['lon_from'] = test.lon_from.str.split(':').str

test = test.drop(['lat_long_from'], axis = 1)

test['lat_to'], test['lon_to'] = test.lat_long_to.str.split(',').str

_, test['lat_to'] = test.lat_to.str.split(':').str

_, test['lon_to'] = test.lon_to.str.split(':').str

test = test.drop(['lat_long_to'], axis = 1)

test = test.drop(['combined'], axis = 1)
# test = preprocessing.scale(test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [79]:
X = X.drop(['combined'], axis = 1)

In [80]:
X.transpose().head(30)

,0,1,2,3,4,5,6,7,8,9,...,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099
Street,15,18,24,24,24,12,17,17,12,24,...,18,18,24,24,24,24,12,12,13,13
From,1,16,17,8,13,10,30,43,37,52,...,21,21,52,19,52,52,49,49,51,51
To,1,42,40,21,46,46,45,55,23,9,...,16,16,20,55,24,9,32,32,17,17
Street.Length,179.133,52.7402,52.5178,52.4053,52.1912,52.3242,146.417,139.494,52.1883,39.028,...,58.041,58.041,40.6716,63.262,44.8157,39.028,48.5558,48.5558,104.83,104.83
from_to,1,22,23,10,17,13,43,62,53,76,...,28,28,78,26,79,76,71,71,74,74
occ_per_1,0.551448,0.435176,0.353977,0.353977,0.353977,0.388997,0.384777,0.384777,0.388997,0.353977,...,0.435176,0.435176,0.353977,0.353977,0.353977,0.353977,0.388997,0.388997,0.427939,0.427939
vac_per_1,0.414499,0.473185,0.645017,0.645017,0.645017,0.454687,0.469562,0.469562,0.454687,0.645017,...,0.473185,0.473185,0.645017,0.645017,0.645017,0.645017,0.454687,0.454687,0.444357,0.444357
occ_per_2,0.673388,0.617462,0.447894,0.447894,0.447894,0.474042,0.470001,0.470001,0.474042,0.447894,...,0.617462,0.617462,0.447894,0.447894,0.447894,0.447894,0.474042,0.474042,0.488741,0.488741
vac_per_2,0.321598,0.349815,0.55098,0.55098,0.55098,0.369394,0.440239,0.440239,0.369394,0.55098,...,0.349815,0.349815,0.55098,0.55098,0.55098,0.55098,0.369394,0.369394,0.382045,0.382045
occ_per_3,0.499,0.409021,0.357339,0.357339,0.357339,0.373845,0.341198,0.341198,0.373845,0.357339,...,0.409021,0.409021,0.357339,0.357339,0.357339,0.357339,0.373845,0.373845,0.347037,0.347037


In [82]:
X = X.fillna(0)
test = test.fillna(0)

In [83]:
scaler = preprocessing.MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
test_scaled = scaler.transform(test)

In [96]:
np.save('tmp/X_scaled', X_scaled)
np.save('tmp/test_scaled',test_scaled)

In [84]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, Y, test_size = .2, random_state=25)

In [85]:
%matplotlib inline
from matplotlib import pyplot as plt

C_param_range = [0.001,0.01,0.1,1,10,100]

acc_table = pd.DataFrame(columns = ['C_parameter','Accuracy'])
acc_table['C_parameter'] = C_param_range

X_combined = np.vstack((X_train,X_val))
Y_combined = np.hstack((y_train, y_val))


plt.figure(figsize=(10, 10))

j = 0
for i in C_param_range:
    
    # Apply logistic regression model to training data
    lr = LogisticRegression(penalty = 'l2', C = i,random_state = 0)
    lr.fit(X_train,y_train)
    
    # Predict using model
    y_pred = lr.predict(X_val)
    
    # Saving accuracy score in table
    acc_table.iloc[j,1] = accuracy_score(y_val,y_pred)
    j += 1
    
acc_table

,C_parameter,Accuracy
0,0.001,0.622727
1,0.010,0.659091
2,0.100,0.727273
3,1.000,0.731818
4,10.000,0.740909
5,100.000,0.736364


In [86]:
# C = 0.1, class_weight='balanced', penalty = 'l1'
LogReg = LogisticRegression(C=10, max_iter=10000, n_jobs=-1,solver='newton-cg',penalty='l2')
LogReg.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='ovr',
          n_jobs=-1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [87]:
yval_pred = LogReg.predict(X_val)
LogReg.predict_proba(X_val)

array([[ 0.55866732,  0.44133268],
       [ 0.93394008,  0.06605992],
       [ 0.72349979,  0.27650021],
       [ 0.93002387,  0.06997613],
       [ 0.9325384 ,  0.0674616 ],
       [ 0.70527307,  0.29472693],
       [ 0.80269611,  0.19730389],
       [ 0.3258618 ,  0.6741382 ],
       [ 0.38099674,  0.61900326],
       [ 0.51732116,  0.48267884],
       [ 0.13683451,  0.86316549],
       [ 0.88388355,  0.11611645],
       [ 0.48301444,  0.51698556],
       [ 0.51074357,  0.48925643],
       [ 0.55317657,  0.44682343],
       [ 0.16958159,  0.83041841],
       [ 0.89521354,  0.10478646],
       [ 0.87680447,  0.12319553],
       [ 0.89186972,  0.10813028],
       [ 0.84354418,  0.15645582],
       [ 0.2910385 ,  0.7089615 ],
       [ 0.45151608,  0.54848392],
       [ 0.04783427,  0.95216573],
       [ 0.82674651,  0.17325349],
       [ 0.5988164 ,  0.4011836 ],
       [ 0.40978191,  0.59021809],
       [ 0.73656441,  0.26343559],
       [ 0.4372691 ,  0.5627309 ],
       [ 0.51780169,

In [88]:
sum(LogReg.predict(X_val) ==1)

70

In [89]:
sum(y_val == 1)

83

In [90]:
from sklearn.metrics import *

In [91]:
fbeta_score(beta=0.5, y_pred=LogReg.predict(X_val), y_true=y_val)

0.66115702479338856

In [92]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, LogReg.predict(X_val))
cm

array([[115,  22],
       [ 35,  48]])

In [93]:
print('Precision: ' + str(np.round(100*float((cm[1][1]))/float((cm[1][1]+cm[0][1])),2))+'%')
print('Recall: ' + str(np.round(100*float((cm[1][1]))/float((cm[1][0]+cm[1][1])),2))+'%')

Precision: 68.57%
Recall: 57.83%


In [98]:
prediction = {"id":range(1,727), "any_spot":LogReg.predict(test_scaled)}

sub_rf_v5 = pd.DataFrame(prediction)

sub_rf_v5 = sub_rf_v5.astype("int")

sub_rf_v5.to_csv("sub_logistic_reg_0306Tina.csv", index =False)